In [ ]:
# Basic Imports
import os
import sys
import warnings
import numpy as  np
import pandas as pd
from scipy import linalg
import random 
from glob import glob

# Features
from sklearn.discriminant_analysis import _class_means,_class_cov
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import pairwise_distances 
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, average_precision_score, precision_recall_curve, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import yaml
from sklearn.metrics import cohen_kappa_score
from scipy import stats
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt


In [ ]:
from sklearn import datasets, svm

In [ ]:
melanoma_dir="../input/melanoma/datasets/datasets/edra_cropped/ready"
retina_dir="../input/melanoma/datasets/datasets/retina"
git_repo_path="../input/melanoma/melanoma-transfer/melanoma-transfer"

In [ ]:
def get_image_files(datadir, listImages=[], left_only=False):
    fs = glob('{}/*'.format(datadir))
    if left_only:
        fs = [f for f in fs if 'left' in f]
    if listImages != []:                                                        # <-- Adapted to return images from the list
        image_name = [img.split(".")[0] for img in listImages]           #listImages contains .tiff and datadir contains .jpg, so extracting image name
        fs = [f for f in fs if f.split("/")[-1].split(".")[0] in image_name]                  # Useful for running with 5x2-fold cross-validation
        print("images listed:",len(listImages)," images loaded:",len(fs))
    return np.array(sorted(fs))

def get_labels(names, labels=None, label_file=None,
               per_patient=False):
    if labels is None:
        labels = pd.read_csv(label_file,
                             index_col=0).loc[names].values.flatten()
    if per_patient:
        left = np.array(['left' in n for n in names])
        return np.vstack([labels[left], labels[~left]]).T
    else:
        return labels
    
def load_features(fnames, test=False):

    if test:
        fnames = [os.path.join(os.path.dirname(f),
                               os.path.basename(f).replace('train', 'test'))
                  for f in fnames]

    data = [np.load(f) for f in fnames]
    data = [X.reshape([X.shape[0], -1]) for X in data]
    return np.hstack(data)

In [ ]:
def estimator(protocol, classifier, n_features, files, X, labels, fold, eval_size=0.1):
    svm = SVC(kernel='linear', class_weight='balanced', cache_size=5500, probability=True)
    if protocol != 'protocol3':
        svm_model = svm
        param_grid = {"C": [1e-3,1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]}
        cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
        est = GridSearchCV(svm_model, param_grid=param_grid, scoring='roc_auc', n_jobs=15, cv=cv, verbose=2)
        est.fit(X, labels.reshape((labels.shape[0],)))
    else:
        param_grid = {"estimator__C": [1e-3,1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]}
        binarized_labels = label_binarize(np.squeeze(labels), classes=[0,1,2])
        svm_model = OneVsRestClassifier(svm)
        cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
        est = GridSearchCV(svm_model, param_grid=param_grid, scoring='roc_auc', n_jobs=15, cv=cv, verbose=2)
        est.fit(X, binarized_labels)
    est = est.best_estimator_
    print("Best estimator found by grid search for %s: " % (classifier))
    print(est)
    
    return est

In [ ]:
def calc_auc(y_pred_proba, labels, exp_run_folder, classifier, fold):
    
    auc = roc_auc_score(labels, y_pred_proba)
    fpr, tpr, thresholds = roc_curve(labels, y_pred_proba)
    curve_roc = np.array([fpr, tpr])
    dataile_id = open(exp_run_folder+'/data/roc_{}_{}.txt'.format(classifier, fold), 'w+')
    np.savetxt(dataile_id, curve_roc)
    dataile_id.close()
    plt.plot(fpr, tpr, label='ROC curve: AUC={0:0.2f}'.format(auc))
    plt.xlabel('1-Specificity')
    plt.ylabel('Sensitivity')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.grid(True)
    plt.title('ROC Fold {}'.format(fold))
    plt.legend(loc="lower left")
    plt.savefig(exp_run_folder+'/data/roc_{}_{}.pdf'.format(classifier, fold), format='pdf')
    return auc


In [ ]:
def fit(features_path,protocol,fold,dataset_dir,dest_path):
    
    #Loading dataset
    classifier="SVM"
    folds = yaml.full_load(open(git_repo_path+'/folds/'+protocol+'.yml'))
    f0, f1 = fold.split('x')
    train_list = folds['Fold_' + f0][int(f1)-1]
    test_list  = folds['Fold_' + f0][0 if f1=='2' else 1]
    image_files = get_image_files(dataset_dir, train_list)
    names = [os.path.basename(x).split('.')[0] for x in image_files]
    labels = get_labels(names, label_file=git_repo_path+'/folds/'+protocol+'.csv').astype(np.int32)[:, np.newaxis]

    y_preds = []
    y_preds_proba = []
    #Loading train and test features
    test_features_path=os.path.join(features_path,"test_features.npy")
    train_features_path=os.path.join(features_path,"train_features.npy")
    print(train_features_path)
    X_train=load_features([train_features_path])
    l2Norm = np.linalg.norm(X_train,axis=1)
    X_train = np.divide(X_train.T,l2Norm).T
    print("Transformed X_train:", X_train.shape)
    est = estimator(protocol, "SVM", X_train.shape[1], image_files, X_train, labels, fold, eval_size=0.1)
    X_test = load_features([test_features_path], test=True)
    l2Norm = np.linalg.norm(X_test,axis=1)
    X_test = np.divide(X_test.T,l2Norm).T
    print("Transformed X_test:", X_test.shape)

    if protocol != 'protocol3':
        y_pred = est.predict(X_test).ravel()
        y_pred_proba = est.predict_proba(X_test).ravel()
        y_proba = []
        for i in range(0, 2*len(X_test), 2):
            y_proba.append(y_pred_proba[i+1]) #using score from the positive
    else:
        y_pred_binary = est.predict(X_test)
        y_pred = preprocessing.LabelBinarizer().fit([0,1,2])
        y_pred = y_pred.inverse_transform(y_pred_binary)
        y_proba = est.predict_proba(X_test)

    image_files = get_image_files(dataset_dir, test_list)
    names = [os.path.basename(x).split('.')[0] for x in image_files]
    labels = get_labels(names, label_file=git_repo_path+'/folds/'+protocol+'.csv').astype(np.int32)[:, np.newaxis]   # , per_patient=per_patient

    image_column = pd.Series(names, name='image')
    labels_column = pd.Series(np.squeeze(labels), name='true')

    level_column = pd.Series(y_pred, name='pred')
    if protocol != 'protocol3':
        proba_column = pd.Series(y_proba, name='proba')
        predictions = pd.concat([image_column, labels_column, level_column, proba_column], axis=1)
    else:
        proba_label_0 = pd.Series(y_proba[:,0], name='proba_label_0')
        proba_label_1 = pd.Series(y_proba[:,1], name='proba_label_1')
        proba_label_2 = pd.Series(y_proba[:,2], name='proba_label_2')
        predictions = pd.concat([image_column, labels_column, level_column, proba_label_0, proba_label_1, proba_label_2], axis=1)
    

    predictions.to_csv(dest_path+"/ranked_list_fold_{}.csv".format(fold), sep=';')
    
    print("tail of predictions")
    print(predictions.tail())
    acc=len([idx for idx in range(len(labels)) if labels[idx]==y_pred[idx]])/float(len(labels))
    #acc = len(list(filter(lambda l,y : l == y, zip(labels, y_pred))))/float(len(labels)) 
    print("accuracy: {}".format(acc))
    print("confusion matrix")
    print(confusion_matrix(labels, y_pred))
    
    if protocol != 'protocol3':
        auc = calc_auc(y_proba, labels, dest_path, classifier, fold)
        print("AUC: {}".format(auc))
        average_precision = average_precision_score(labels, y_proba)
        print("average precision: {}".format(average_precision))
        c_matrix = confusion_matrix(labels, y_pred)
        print("sensitivity: {}".format(c_matrix[1][1]/(c_matrix[1][1]+ c_matrix[0][1])))
        print("specificity: {}".format(c_matrix[0][0]/(c_matrix[0][0]+ c_matrix[1][0])))
    else:
        y_test = label_binarize(labels, classes=[0, 1, 2])
        auc = roc_auc_score(y_test, y_proba, average='macro')
        print("AUC: {}".format(auc))
        average_precision = average_precision_score(y_test, y_proba, average="macro")
        print("mean average precision: {}".format(average_precision))    

    results = pd.concat([pd.Series(dest_path,name='folder'), pd.Series(fold,name='fold'), pd.Series(auc,name='auc'), pd.Series(average_precision,name='ap'), pd.Series(acc,name='acc')],axis=1)
    with open('results.csv', 'a') as f:
        results.to_csv(f, header=False)



In [ ]:
def main(protocol,model_name,directory,dataset_dir):
    source_path=os.path.join(directory,model_name+"_"+protocol+"_features")
    result_path="./"+model_name+"_"+protocol
    os.mkdir(result_path)
    os.mkdir(result_path+"/data")
    for i in range(1,6):
        for j in range(1,3):
            fold=str(i)+"x"+str(j)
            feature_path=os.path.join(source_path,fold)
            fit(feature_path,protocol,fold,dataset_dir,result_path)

In [ ]:
main("protocol1","E","../input/melanoma-features",melanoma_dir)